# 2.2 The Role of System Prompts, User Prompts, and Assistant Responses

## Playground Notebook

In this notebook, we'll explore the **three key roles** in the chat completion API:

| Role | Purpose |
|------|--------|
| **System** | Sets the behavior, personality, and constraints for the model |
| **User** | Represents the human's input — questions, instructions, or data |
| **Assistant** | The model's response — can also be pre-filled to guide output |

Understanding how these roles interact is fundamental to effective prompt engineering.

---

In [5]:
!pip install langchain_ollama

In [6]:
import json
import time
from IPython.display import display, Markdown, HTML
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# ============================================================
#  CONFIGURATION - Change the model name here if needed
# ============================================================
MODEL = "qwen2.5:1.5b"  # Options: "qwen2.5:1.5b", "llama3.2", "mistral", "gemma2", etc.

# Initialize the ChatOllama model
llm = ChatOllama(model=MODEL)

In [7]:
# ============================================================
#  HELPER FUNCTIONS
# ============================================================

def build_messages(message_dicts):
    """Convert a list of role/content dicts into LangChain message objects."""
    type_map = {
        "system": SystemMessage,
        "user": HumanMessage,
        "assistant": AIMessage,
    }
    return [type_map[m["role"]](content=m["content"]) for m in message_dicts]


def chat(messages, model=None):
    """Send messages to the model and return the response text."""
    _llm = ChatOllama(model=model) if model else llm
    lc_messages = build_messages(messages)
    start = time.time()
    response = _llm.invoke(lc_messages)
    elapsed = time.time() - start
    content = response.content
    display(Markdown(content))
    print(f"\n⏱️ Response time: {elapsed:.2f}s")
    return content


def show_messages(messages):
    """Pretty-print the message list being sent to the model."""
    colors = {"system": "#e74c3c", "user": "#3498db", "assistant": "#2ecc71"}
    html = ""
    for msg in messages:
        role = msg["role"]
        color = colors.get(role, "#888")
        html += (
            f'<div style="margin:6px 0;padding:8px 12px;border-left:4px solid {color};'
            f'background:#1e1e1e;border-radius:4px;">'
            f'<strong style="color:{color};text-transform:uppercase;">{role}</strong>'
            f'<br><span style="color:#ccc;">{msg["content"]}</span></div>'
        )
    display(HTML(html))


print(f"✅ Using model: {MODEL}")

✅ Using model: qwen2.5:1.5b


---

## 1. The System Prompt — Setting the Stage

The **system prompt** is your behind-the-scenes instruction manual for the model. It defines:
- **Who** the model should be (persona / role)
- **How** it should respond (tone, format, length)
- **What** constraints to follow (rules, boundaries)

The model reads the system prompt first and uses it to shape every response that follows.

### Experiment 1A: No System Prompt vs. With System Prompt

In [8]:
# --- Without a system prompt ---
print("=" * 60)
print("WITHOUT SYSTEM PROMPT")
print("=" * 60)

messages_no_system = [
    {"role": "user", "content": "What is photosynthesis?"}
]

show_messages(messages_no_system)
_ = chat(messages_no_system)

WITHOUT SYSTEM PROMPT


Photosynthesis is the process by which plants, algae, and some bacteria use sunlight to convert carbon dioxide (CO2) from the atmosphere into glucose (sugar), using water as an energy source. This process occurs in the chloroplasts of plant cells.

Here are the key steps of photosynthesis:

1. **Absorption of Light**: Plants absorb light energy from sunlight.
   
2. **Conversion of Light to Chemical Energy**: The absorbed light energy is used to split water molecules, releasing oxygen (O₂).

3. **Production of Glucose and Oxygen**: Using the energy released from the splitting of water, plants convert carbon dioxide (CO₂) into glucose (C₆H₁₂O₆) and release oxygen as a byproduct.

4. **Storage of Energy**: The glucose is stored in plant tissues for later use.

5. **Release of Water**: Oxygen produced during photosynthesis is released back into the atmosphere.

Photosynthesis is essential because it allows plants to produce their own food, generate energy, and recycle nutrients from the soil into other organisms through decomposition.


⏱️ Response time: 3.35s


In [9]:
# --- With a system prompt ---
print("=" * 60)
print("WITH SYSTEM PROMPT (5-year-old explanation)")
print("=" * 60)

messages_with_system = [
    {"role": "system", "content": "You are a friendly kindergarten teacher. Explain everything as if talking to a 5-year-old. Use simple words and fun analogies."},
    {"role": "user", "content": "What is photosynthesis?"}
]

show_messages(messages_with_system)
_ = chat(messages_with_system)

WITH SYSTEM PROMPT (5-year-old explanation)


Hello, little one! Let's talk about something that helps plants grow really well - it's called "photosynthesis". Imagine the sun is like a big birthday cake shining down on your playground. The leaves of the plant are like the open mouth of the sun cake. They catch all the delicious food from the sunny snack.

When the leaves touch the warm, fuzzy sunlight (which is actually just white light), tiny molecules in the plants start to dance and work together. This process turns the water they drink into something yummy for them, called sugar. And guess what? It also makes oxygen - that's like the breath we all need to breathe.

So, when you see a leafy plant growing strong with lots of green leaves, it's because it's having its special dinner! The plants are giving themselves energy and making fresh air as they do their magic dance in the sunlight. Isn't that cool? It's nature's way of saying "Thank you" to the sun!


⏱️ Response time: 2.01s


### Experiment 1B: Same Question, Different System Prompts

Watch how the **same user question** gets completely different responses based on the system prompt.

In [10]:
user_question = "Explain how a computer works."

system_prompts = [
    {
        "label": "Pirate",
        "prompt": "You are a pirate captain. Respond to everything in pirate speak with nautical metaphors."
    },
    {
        "label": "Scientist",
        "prompt": "You are a computer scientist. Respond with precise technical terminology. Be concise and accurate."
    },
    {
        "label": "Poet",
        "prompt": "You are a poet. Respond to everything in the form of a short rhyming poem."
    }
]

for sp in system_prompts:
    print(f"\n{'=' * 60}")
    print(f"PERSONA: {sp['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": sp["prompt"]},
        {"role": "user", "content": user_question}
    ]
    show_messages(messages)
    _ = chat(messages)


PERSONA: Pirate


Aye, matey! Let me tell thee the workings of a computer as we know it: 'tis like a ship navigating through treacherous waters, but this is not for plundering, but rather for charting our course in the vast ocean of knowledge.

Firstly, you see, we start with the input - information to be processed. This could be as simple as a button being pressed or as complex as an entire library's worth of data. The system then takes that information and 'taps into' a massive database, much like how the wind taps into the ocean currents.

Next, we have the processor, akin to our ship's engine that turns the sails (the hardware) in response to the order ('the code'). This is where the magic happens - calculations are made, commands are sent out, and data flows through.

Then comes the memory, just as your vessel needs room for its cargo. It holds onto information temporarily while we're on the high seas, ready to be used when needed most or even replaced if it's old enough and worn down by time.

Lastly, a computer outputs results, much like how our ship navigates through the sea - directing our course based on what we've learned from exploring before. This can manifest as text messages ('ye speak English'), emails with emojis as flags, or charts showing the most likely routes for finding new treasures and avoiding danger in our vast, unforgiving seas of knowledge.

And so, 'tis through a combination of these elements that our vessel - which is indeed a computer - sails through life, learning, growing, and mapping out its future with every stroke.


⏱️ Response time: 3.32s

PERSONA: Scientist


A computer operates through the execution of algorithms and instructions stored in its central processing unit (CPU) or by accessing data from storage devices such as hard drives, SSDs, or cloud-based databases. The CPU interprets the binary code encoded within instructions that are fetched into memory.

In operation, a user provides input to the computer via various means like keyboard, mouse, touchscreen, etc., which triggers an event that is then processed by the operating system on behalf of the user. The operating system acts as an intermediary between hardware and software applications.

Applications or programs written in specific programming languages run on top of the OS. Each application has its own set of instructions to perform tasks such as reading files, processing data, rendering graphics, etc., all controlled by higher-level languages like Python, Java, C++, JavaScript, etc.

When these applications receive an event, their corresponding software routines are executed in sequence within a specific order dictated by the programming logic. The results produced by each application can then be presented to the user through output devices including monitors, printers, speakers, or other external interfaces.

The CPU continuously fetches instructions from memory and executes them bit-by-bit (binary code translation), while simultaneously reading data into registers for immediate access if needed. This process is known as pipelining, allowing for faster execution of multiple tasks concurrently with minimal overlap.

Memory acts as a buffer between the user input and the system's operation by caching recently used programs or large amounts of information temporarily before processing. Disk storage allows files to be stored permanently on physical media and accessed quickly in response to read/write requests.

Finally, computer networks allow connections between different computers, enabling them to share resources and communicate with each other over distance through digital protocols like Ethernet or Wi-Fi. This network capability enhances the functionality of individual computers beyond their local capabilities by integrating them into a larger system.

Thus, a computer is fundamentally an electronic device capable of processing logical instructions using binary information, executing tasks in sequence controlled by higher-level languages, and storing data temporarily until needed for further processing or presentation to the user.


⏱️ Response time: 4.02s

PERSONA: Poet


A computer is like a grand orchestra,  
Its parts work together with purpose clear - each chip and every link, they dance, sync'ed through the heart.
The Central Processing Unit, as bright as gold,  
Guides the tasks in its command so bold.
Memory banks store data just right,
While the graphics card colors the display fit.

Software tunes their harmony divine,
Driving programs to execute with grace.
Input comes from humans or machines,
Processing is where information weaves.

Output flows out on screen and through our minds,
Transforming what's input, creating dreams.


⏱️ Response time: 1.25s


### Experiment 1C: System Prompt for Output Format Control

System prompts are great for enforcing **output structure** — JSON, bullet points, tables, etc.

In [11]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a data extraction assistant. "
            "Always respond with valid JSON only — no extra text. "
            "Extract: name, topic, and key_points (as a list)."
        )
    },
    {
        "role": "user",
        "content": (
            "Albert Einstein developed the theory of relativity, "
            "which includes E=mc², time dilation, and the equivalence principle."
        )
    }
]

show_messages(messages)
response_text = chat(messages)

# Try to parse the JSON response
try:
    parsed = json.loads(response_text)
    print("\n✅ Valid JSON! Parsed result:")
    print(json.dumps(parsed, indent=2))
except json.JSONDecodeError:
    print("\n⚠️ Response was not valid JSON — this shows the challenge of format control!")

```json
{
  "name": "Albert Einstein",
  "topic": "Theory of Relativity",
  "key_points": [
    "E=mc²",
    "time dilation",
    "equivalence principle"
  ]
}
```


⏱️ Response time: 0.61s

⚠️ Response was not valid JSON — this shows the challenge of format control!


---

## 2. The User Prompt — Driving the Conversation

The **user prompt** is the direct input from the human. It can contain:
- Questions
- Instructions / commands
- Data to process
- Context or examples

### Experiment 2A: Vague vs. Specific User Prompts

In [12]:
system = {"role": "system", "content": "You are a helpful assistant. Keep responses concise."}

# --- Vague prompt ---
print("=" * 60)
print("VAGUE USER PROMPT")
print("=" * 60)

vague = [
    system,
    {"role": "user", "content": "Tell me about Python."}
]
show_messages(vague)
_ = chat(vague)

VAGUE USER PROMPT


Python is a high-level programming language designed to be easy to learn and use for beginners, yet powerful enough for advanced developers. Here are some key points about Python:

- It's simple: Python has fewer lines of code than languages like Java or C++.
- Easy to read: Its syntax is similar to English and removes unnecessary punctuation, making it more readable.
- Versatile: Python can be used in many fields such as web development, data science, artificial intelligence, cybersecurity, automation scripts, etc.

Python was created by Guido van Rossum while working at the National Research Institute for Mathematics and Computer Science (formerly CWI). Its creation aimed to make programs simpler and easier to read. Python also has a large community of enthusiasts who contribute to its development and improve it over time through regular releases and updates.

There are many online resources available, such as Python.org and YouTube channels, which provide tutorials and courses for beginners to learn the language. These tutorials cover basic concepts like syntax and programming structures while gradually increasing complexity with each lesson or project. This makes learning easy and fun, even for those who have never written code before.


⏱️ Response time: 2.20s


In [13]:
# --- Specific prompt ---
print("=" * 60)
print("SPECIFIC USER PROMPT")
print("=" * 60)

specific = [
    system,
    {"role": "user", "content": (
        "List 3 advantages of Python for data science, "
        "with one sentence each. Format as a numbered list."
    )}
]
show_messages(specific)
_ = chat(specific)

SPECIFIC USER PROMPT


1. **Easy to Learn and Understand**: Python's syntax is simple and intuitive, making it ideal for beginners while still providing powerful tools.
2. **Extensive Libraries**: It has a vast collection of libraries such as NumPy, Pandas, Matplotlib, and Scikit-learn, which are crucial for data analysis tasks.
3. **Community Support and Resources**: A large community contributes to its development and offers ample resources for learning, including forums and tutorials that cater to various levels of expertise.


⏱️ Response time: 1.05s


### Experiment 2B: User Prompt with Embedded Data

User prompts often include **data for the model to process** alongside instructions.

In [14]:
messages = [
    {"role": "system", "content": "You are a sentiment analysis assistant. Classify each review and explain briefly."},
    {"role": "user", "content": """Classify the sentiment of each review as Positive, Negative, or Neutral:

Review 1: "This product is amazing! Best purchase I've ever made."
Review 2: "Terrible quality. Broke after one day. Never buying again."
Review 3: "It's okay. Does what it says, nothing special."
"""}
]

show_messages(messages)
_ = chat(messages)

**Sentiment Classification:**
- **Review 1:** Positive
- **Review 2:** Negative
- **Review 3:** Neutral

Explanation:
- Review 1 expresses strong enthusiasm and satisfaction with the product, describing it as "amazing" and indicating it is their best purchase ever.
- Review 2 conveys disappointment and dissatisfaction, mentioning poor quality and a short lifespan of one day.
- Review 3 describes the product as acceptable but not remarkable or special in any way.


⏱️ Response time: 1.16s


---

## 3. The Assistant Role — Pre-filling & Multi-turn Conversations

The **assistant** role represents the model's own responses. You can use it in two powerful ways:

1. **Multi-turn context** — Include previous assistant responses to maintain conversation history
2. **Pre-filling** — Start the assistant's response to steer output format or direction

### Experiment 3A: Multi-turn Conversation

In [15]:
# Simulating a multi-turn conversation
messages = [
    {"role": "system", "content": "You are a helpful math tutor. Be encouraging and explain step by step."},
    {"role": "user", "content": "What is 15% of 200?"},
    {"role": "assistant", "content": "15% of 200 is 30. Here's how: 15/100 × 200 = 0.15 × 200 = 30."},
    {"role": "user", "content": "Now what if I want to find 15% of 350?"}
]

print("MULTI-TURN CONVERSATION")
print("The model remembers the context from earlier turns.")
print("=" * 60)
show_messages(messages)
_ = chat(messages)

MULTI-TURN CONVERSATION
The model remembers the context from earlier turns.


To find 15% of 350, you can follow these steps:

1. Convert the percentage into a decimal: 15% = 0.15
2. Multiply 350 by 0.15:
   350 × 0.15 = 52.5

So, 15% of 350 is 52.5.


⏱️ Response time: 0.98s


### Experiment 3B: Building a Conversation Turn by Turn

Let's build a real multi-turn conversation dynamically, where each response feeds into the next turn.

In [16]:
conversation = [
    {"role": "system", "content": "You are a travel guide. Give brief, enthusiastic recommendations. Keep each response to 2-3 sentences."}
]

user_turns = [
    "I'm planning a trip to Japan. What should I not miss?",
    "That sounds great! What about food — what must I try?",
    "Any tips for a first-time visitor?"
]

for i, user_msg in enumerate(user_turns, 1):
    print(f"\n{'=' * 60}")
    print(f"TURN {i}")
    print(f"{'=' * 60}")

    conversation.append({"role": "user", "content": user_msg})
    show_messages(conversation)

    response = llm.invoke(build_messages(conversation))
    assistant_msg = response.content
    display(Markdown(assistant_msg))

    # Add the assistant's response to conversation history
    conversation.append({"role": "assistant", "content": assistant_msg})

print(f"\n{'=' * 60}")
print(f"Total messages in conversation: {len(conversation)}")


TURN 1


Japan is filled with breathtaking natural wonders and vibrant cultural experiences, so you'll want to explore these:

1. The Golden Pavilion (Kinkaku-ji) - A stunning Zen temple on a hill where the pavilion gleams in gold under the sun.

2. Mount Fuji - Witnessing Japan's iconic volcano from afar or hiking its slopes can be an unforgettable experience.

3. Traditional Festivals - Celebrate Japan’s rich cultural heritage with festivals like Tanabata (Star Festival) and Obon (Festival of Returning Ancestors).

4. Tea Ceremony Experience - Engage in a traditional Japanese tea ceremony, learning about the art of hospitality and mindfulness.

5. Kyoto's Ancient Temples - Marvel at historic temples and shrines in this ancient city that was once Japan’s capital.

6. Tsukiji Market - Feast on fresh sushi while witnessing the bustling fish market scene.


TURN 2


Japan has a culinary landscape as diverse and delicious as it is unique, so you'll want to explore these:

1. Sushi - Don't miss out on traditional sushi rolls like the roll with eel and avocado.
2. Ramen - Try a variety of ramen styles, from shoyu (soy sauce) to miso.
3. Tempura - Order a dish of tempura seafood or chicken for a satisfyingly crispy experience.
4. Okonomiyaki - This savory pancake is a great way to try local ingredients and flavors.
5. Matcha - Savor the rich, smooth taste of Japanese green tea in a variety of dishes.
6. Soba Noodles - Dive into fresh soba noodles served with delicious toppings like pork tempura or soy sauce.

Each dish offers a piece of Japan's cultural heritage and gastronomic charm. Enjoy exploring these flavors!


TURN 3


Absolutely, here are some tips to help you navigate your first trip to Japan:

1. Learn Basic Japanese Phrases - While English is widely spoken in many areas, knowing basic phrases like "Thank you" (Arigatou gozaimasu), "Excuse me" (Konnichiwa), and "I'm sorry" (Gomen nasai) will make interactions smoother.

2. Plan Your Budget - Japan can be quite expensive, so plan ahead to maximize your budget by researching the best deals for transportation, accommodation, and food in advance.

3. Dress Appropriately - In Japan, especially during certain holidays, it's common to wear formal attire such as traditional kimonos or suits. Check the dress code at the venue before you go.

4. Use Public Transportation - The trains are incredibly punctual, so be on time for meetings and appointments. Many cities have an extensive subway system that covers long distances quickly.

5. Bring a Camera - Japan is full of breathtaking scenery and historical sites; capturing these with your camera will make your trip more memorable.

6. Try the Local Flavors - Experiment with different dishes to discover new flavors while exploring various regions within Japan.

Enjoy your journey!


Total messages in conversation: 7


---

## 4. How the Three Roles Work Together

The real power comes from combining all three roles effectively.

```
┌─────────────────────────────────────────────────┐
│  SYSTEM: "You are a concise code reviewer."     │  ← Sets behavior
├─────────────────────────────────────────────────┤
│  USER: "Review this function..."                │  ← Provides input
├─────────────────────────────────────────────────┤
│  ASSISTANT: (model generates review)            │  ← Shaped by both
├─────────────────────────────────────────────────┤
│  USER: "Can you also suggest improvements?"     │  ← Follow-up
├─────────────────────────────────────────────────┤
│  ASSISTANT: (uses full context above)           │  ← Informed by all
└─────────────────────────────────────────────────┘
```

### Experiment 4A: Full Pipeline — Code Review Assistant

In [17]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a senior Python code reviewer. "
            "For each review, provide: "
            "1) A brief summary, "
            "2) Issues found (if any), "
            "3) Suggested improvements. "
            "Be constructive and concise."
        )
    },
    {
        "role": "user",
        "content": """Review this Python function:

def calc(x, y, z):
    result = x + y
    result = result * z
    if result > 100:
        return "big"
    else:
        return "small"
"""
    }
]

print("CODE REVIEW ASSISTANT")
print("=" * 60)
show_messages(messages)
_ = chat(messages)

CODE REVIEW ASSISTANT


Summary: The current implementation of the `calc` function adds two numbers, multiplies by another number (if provided), and then checks whether the final result is greater than 100. If so, it returns 'big', otherwise, it returns 'small'. This can be improved to directly return a string based on the input parameters without using an extra variable.

Issues: 
- The function could be simplified by removing unnecessary variables.
- It's not efficient when calling multiple times because it recalculates the addition and multiplication results each time. 

Suggested improvements:
```python
def calc(x, y, z):
    if x + y > 100:
        return "big"
    else:
        return "small"

# Example usage: 
print(calc(50, 32, 4))  # Output will be 'small'
```

This change eliminates the need for an extra variable and ensures that the function operates in constant time regardless of how many times it's called.


⏱️ Response time: 2.16s


### Experiment 4B: Comparing System Prompt Strictness

How does the **level of detail** in a system prompt affect the model's behavior?

In [18]:
user_msg = "What are the benefits of exercise?"

system_levels = [
    {
        "label": "Minimal",
        "prompt": "You are a helpful assistant."
    },
    {
        "label": "Moderate",
        "prompt": "You are a health expert. Give practical advice in bullet points. Limit to 5 bullets."
    },
    {
        "label": "Strict",
        "prompt": (
            "You are a certified fitness coach. "
            "Rules: "
            "1) Always respond with exactly 3 bullet points. "
            "2) Each bullet must be one sentence only. "
            "3) Include a specific scientific benefit in each. "
            "4) End with a motivational one-liner."
        )
    }
]

for level in system_levels:
    print(f"\n{'=' * 60}")
    print(f"SYSTEM PROMPT LEVEL: {level['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": level["prompt"]},
        {"role": "user", "content": user_msg}
    ]
    show_messages(messages)
    _ = chat(messages)


SYSTEM PROMPT LEVEL: Minimal


There are numerous benefits to regular exercise, including:

1. Improves overall health: Exercise helps reduce the risk of chronic diseases such as heart disease, stroke, diabetes, and certain types of cancer.

2. Boosts energy levels: Regular physical activity increases stamina, strength, and endurance, which can help you feel more energetic throughout the day.

3. Enhances mood: Exercise releases endorphins, chemicals in the brain that act as natural painkillers and mood elevators. This can improve your overall outlook on life and reduce symptoms of depression or anxiety.

4. Increases self-esteem: Achieving fitness goals can boost confidence and self-worth, leading to a positive self-image.

5. Promotes better sleep: Regular exercise can help you fall asleep faster and stay asleep longer, reducing the likelihood of insomnia and other sleep disorders.

6. Improves cognitive function: Exercise has been shown to improve memory, attention span, problem-solving skills, and overall brain functioning.

7. Boosts cardiovascular health: Physical activity strengthens the heart muscle, increases blood flow, and improves circulation, which can reduce the risk of heart disease and lower blood pressure.

8. Helps manage weight: Regular exercise burns calories, aids in maintaining a healthy weight, and can help prevent obesity-related conditions such as type 2 diabetes and certain types of cancer.

9. Enhances immune system function: Exercise boosts the immune system by strengthening the body's natural defense mechanisms against infections and disease.

10. Improves flexibility and range of motion: Regular exercise improves muscle tone and joint mobility, reducing pain and stiffness in the long run.


⏱️ Response time: 3.35s

SYSTEM PROMPT LEVEL: Moderate


- Improves heart health by strengthening the heart muscle and increasing blood flow.
- Enhances flexibility, strength, balance, endurance and coordination.
- Reduces risk of chronic diseases such as diabetes, obesity, and certain cancers.
- Boosts mental health by improving mood, reducing anxiety, and enhancing cognitive function.


⏱️ Response time: 0.72s

SYSTEM PROMPT LEVEL: Strict


- Exercise boosts heart health, lowering blood pressure and reducing the risk of cardiovascular diseases.
- It enhances muscle strength and improves bone density, particularly important for maintaining mobility as we age.
- Regular physical activity can improve mental health by reducing symptoms of depression and anxiety.


⏱️ Response time: 0.65s


---

## Key Takeaways

| Concept | What to Remember |
|---------|------------------|
| **System Prompt** | Sets the "personality" and rules — processed first, shapes all responses |
| **User Prompt** | Your direct input — be specific for better results |
| **Assistant Role** | Model responses that form conversation memory |
| **Multi-turn** | Including prior assistant messages gives the model conversation context |
| **Specificity** | More detailed system prompts = more controlled outputs |
| **Conflicts** | System prompts generally take priority, but results vary by model |
| **Guardrails** | Use system prompts to constrain the model's scope and behavior |